## Prepare SAR offset results from HDF5/GeoTIFF to text for UCLA group

In [28]:
%matplotlib inline
import os
import numpy as np
from matplotlib import pyplot as plt
from mintpy.utils import readfile, writefile, plot as pp, isce_utils, utils as ut
plt.rcParams.update({'font.size': 12})

work_dir = os.path.expanduser('~/data/archives/2021MadoiEQ/offset4MadoiEQ')
os.chdir(work_dir)
print('Go to directory:', work_dir)

fnames = [
    os.path.join(work_dir, 'rdr/S1A099_20210520_20210526_rgOff.h5'),
    os.path.join(work_dir, 'rdr/S1A099_20210520_20210526_azOffCor.h5'),
    os.path.join(work_dir, 'rdr/S1D106_20210520_20210526_rgOff.h5'),
    os.path.join(work_dir, 'rdr/S1D106_20210520_20210526_azOffCorFilt.h5'),
]

Go to directory: /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ


In [30]:
for fname in fnames:
    out_file = os.path.splitext(fname)[0]+'.txt'
    print('-'*50)
    print('reading {}'.format(fname))
    atr = readfile.read_attribute(fname)
    dis = readfile.read(fname, datasetName='displacement')[0]
    mask = ~np.isnan(dis)
    dis = dis[mask]
    print('num of points: {}'.format(np.sum(mask)))

    lat = readfile.read(fname, datasetName='latitude')[0][mask]
    lon = readfile.read(fname, datasetName='longitude')[0][mask]
    dis_std   = readfile.read(fname, datasetName='displacementStd')[0][mask]
    inc_angle = readfile.read(fname, datasetName='incidenceAngle')[0][mask] * np.pi / 180.
    az_angle  = readfile.read(fname, datasetName='azimuthAngle')[0][mask] * np.pi / 180.
    if 'rgOff' in os.path.basename(fname):
        # unit vector for range offset [positive for motion away from satellite]
        ve = np.sin(inc_angle) * np.sin(az_angle)
        vn = np.sin(inc_angle) * np.cos(az_angle) * -1
        vz = np.cos(inc_angle) * -1
    else:
        # unit vector for azimuth offset [positive for motion same as flight]
        ve = np.sin(az_angle)
        vn = np.cos(az_angle) * -1
        vz = np.zeros(az_angle.shape, dtype=np.float32)

    # write to text file
    header =  'number of points: {}\n'.format(np.sum(mask))
    header += 'latitude       longitude    displacement[m]        Vz              Vn              Ve     displacement_std [m]'
    data = np.hstack((
        lat.reshape(-1, 1),
        lon.reshape(-1, 1),
        dis.reshape(-1, 1),
        vz.reshape(-1, 1),
        vn.reshape(-1, 1),
        ve.reshape(-1, 1),
        dis_std.reshape(-1, 1),
    ))
    print('writing to text file: {}'.format(out_file))
    np.savetxt(out_file, data, fmt='%10.6f', delimiter='\t', header=header)
    print('finished writing.')

--------------------------------------------------
reading /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1A099_20210520_20210526_rgOff.h5
num of points: 9139246
writing to text file: /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1A099_20210520_20210526_rgOff.txt
finished writing.
--------------------------------------------------
reading /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1A099_20210520_20210526_azOffCor.h5
num of points: 1785379


/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in sin
/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in cos


writing to text file: /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1A099_20210520_20210526_azOffCor.txt
finished writing.
--------------------------------------------------
reading /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1D106_20210520_20210526_rgOff.h5
num of points: 9070445


/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in sin
/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sin
/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in cos
/Users/yunjunz/tools/miniconda3/envs/insar/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in cos


writing to text file: /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1D106_20210520_20210526_rgOff.txt
finished writing.
--------------------------------------------------
reading /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1D106_20210520_20210526_azOffCorFilt.h5
num of points: 1628546
writing to text file: /Users/yunjunz/data/archives/2021MadoiEQ/offset4MadoiEQ/rdr/S1D106_20210520_20210526_azOffCorFilt.txt
finished writing.
